In [ ]:
import os
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pickle
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler, Subset, Dataset
from sklearn.model_selection import KFold
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, ConfusionMatrixDisplay

PIL.Image.MAX_IMAGE_PIXELS = 933120000000

LR = 5e-4
EPOCHS = 100
BATCH_SIZE = 4
N_SPLITS = 5
N_ACCUMULATE = 16
MODEL_NAME = 'resnext50_32x4d.a1_in1k'
Image_size = 512
WEIGHT_DECAY = 1e-4
num_classes = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())


# //
# //                       _oo0oo_
# //                      o8888888o
# //                      88" . "88
# //                      (| -_- |)
# //                      0\  =  /0
# //                    ___/`---'\___
# //                  .' \\|     |// '.
# //                 / \\|||  :  |||// \
# //                / _||||| -:- |||||- \
# //               |   | \\\  -  /// |   |
# //               | \_|  ''\---/''  |_/ |
# //               \  .-\__  '-'  ___/-. /
# //             ___'. .'  /--.--\  `. .'___
# //          ."" '<  `.___\_<|>_/___.' >' "".
# //         | | :  `- \`.;`\ _ /`;.`/ - ` : | |
# //         \  \ `_.   \_ __\ /__ _/   .-` /  /
# //     =====`-.____`.___ \_____/___.-`___.-'=====
# //                       `=---='
# //
# //
# //     ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# //
# //               佛祖保佑         永无BUG
# //

In [ ]:
# Load the saved .npy files
X_train_path = '/kaggle/input/cs640-numpy/X_train.npy'  # replace with the actual path
Y_train_path = '/kaggle/input/cs640-numpy/Y_train.npy'  # replace with the actual path

# Check if the files exist before loading them
if os.path.exists(X_train_path) and os.path.exists(Y_train_path):
    X_train = np.load(X_train_path)
    Y_train = np.load(Y_train_path)
else:
    print("Files not found. Please check the paths.")

# Verify the shape of arrays
print(f"X_train shape: {X_train.shape}")
print(f"Y_train shape: {Y_train.shape}")

X_train shape: (430, 512, 512, 3)
Y_train shape: (430,)


In [ ]:
print(f"X_train shape: {X_train.shape}")
X_train_tensor = torch.tensor(X_train).float()
Y_train_tensor = torch.from_numpy(Y_train).long()  # assume Y_train contains integer class labels
plt.imshow(X_train_tensor[0]/255)
plt.show()

In [ ]:
X_train_tensor = X_train_tensor.view(-1, Image_size, Image_size, 3)
dataset = TensorDataset(X_train_tensor, Y_train_tensor)
# Split the dataset into a training set and a validation set
train_idx, val_idx = train_test_split(
    np.arange(len(X_train)),
    test_size=0.2,  # reserve 20% for validation
    random_state=43,  # guarantee repeatability
    stratify=Y_train  # keep data distribution consistent
)

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)

train_labels = [dataset.tensors[1][i].item() for i in train_idx]
val_labels = [dataset.tensors[1][i].item() for i in val_idx]

train_images = [dataset[i][0] for i in train_idx]
val_images = [dataset[i][0] for i in val_idx]

plt.imshow(train_images[0]/255)
plt.show()
print(train_images[0].shape)

Y_train = np.array(Y_train)  # convert to numpy array if not already
class_sample_count = np.array([len(np.where(Y_train[train_idx] == t)[0]) for t in np.unique(Y_train[train_idx])])

weights = 1. / class_sample_count
samples_weights = np.array([weights[t] for t in Y_train[train_idx]])

# Convert to a PyTorch tensor
samples_weights = torch.from_numpy(samples_weights)
samples_weights = samples_weights.double()

# Create a sampler for weighted sampling
sampler = WeightedRandomSampler(samples_weights, len(samples_weights))

# Create data loaders for training and validation
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(len(np.unique(Y_train)))

first_batch_images, first_batch_labels = next(iter(train_loader))

plt.imshow(first_batch_images[0]/255)
plt.show()

In [ ]:
image_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), 
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    # transforms.CenterCrop((224, 224)),  
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Display the transformation
for images, labels in train_loader:
    print(images.shape)
    images = images/255
    images = images.permute(0, 3, 1, 2)
    enhanced_image = image_transforms(images)
    print(images.shape)
    enhanced_image = enhanced_image.permute(0, 2, 3, 1)
    plt.imshow(enhanced_image[0])
    plt.show()
    break

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, epochs, device, patience=18):
    model.to(device)

    history = {'train_loss': [], 'val_loss': [], 'train_balanced_acc': [], 'val_balanced_acc': []}

    best_val_acc = 0.0  # keep track of it
    best_val_loss = float('inf')  # keep track of it if needed
    epochs_no_improve = 0  # use early-stopping strategy

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_targets = []

        # Display the training process
        train_loop = tqdm(train_loader, position=0, leave=True)
        for batch_index, (inputs, labels) in enumerate(train_loop):
            inputs, labels = inputs.to(device), labels.to(device)
            inputs = inputs/255
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = image_transforms(inputs)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()

            # Update gradients after every N_ACCUMULATE batches
            if (batch_index + 1) % N_ACCUMULATE == 0:
                optimizer.step()
                optimizer.zero_grad()

            # Record training loss and accuracy
            train_loss += loss.item() # accumulated
            _, predicted = torch.max(outputs, 1)
            train_loop.set_description(f"Epoch [{epoch + 1}/{epochs}]")
            train_preds.extend(predicted.view(-1).cpu().numpy())
            train_targets.extend(labels.view(-1).cpu().numpy())
        
        # Make sure the last accumulated gradient is also updated
        if len(train_loader) % N_ACCUMULATE != 0:
            optimizer.step()
            optimizer.zero_grad()

        # Complete the validation process
        model.eval()
        val_preds = []
        val_targets = []
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                inputs = inputs/255
                inputs = inputs.permute(0, 3, 1, 2)
                inputs = image_transforms(inputs)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.view(-1).cpu().numpy())
                val_targets.extend(labels.view(-1).cpu().numpy())
                val_correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_balanced_acc = balanced_accuracy_score(train_targets, train_preds)
        val_balanced_acc = balanced_accuracy_score(val_targets, val_preds)
        scheduler.step(val_balanced_acc)

        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_balanced_acc'].append(train_balanced_acc)
        history['val_balanced_acc'].append(val_balanced_acc)
        
        if val_balanced_acc > best_val_acc:
            best_val_acc = val_balanced_acc
            epochs_no_improve = 0  # reset
            torch.save(model.state_dict(), f'model_checkpoint.pth')
            print(f"Checkpoint saved at epoch {epoch + 1}")
        else:
            epochs_no_improve += 1
            print(f"No improvement in validation loss for {epochs_no_improve} consecutive epochs.")

            # Stop the training process if the tolerance is reached
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after {epoch + 1} epochs.")
                break

        # Print info of each epoch
        print(f"**Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Train Balanced Acc: {train_balanced_acc:.4f}, Val Loss: {val_loss:.4f}, Val Balanced Acc: {val_balanced_acc:.4f}")

    return history

def plot_metrics(history):
    actual_epochs = len(history['train_loss']) 
    epochs_range = range(1, actual_epochs + 1)
    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, history['train_loss'], label='Train Loss')
    plt.plot(epochs_range, history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Over Time')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, history['train_balanced_acc'], label='Train Accuracy')
    plt.plot(epochs_range, history['val_balanced_acc'], label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Over Time')
    plt.legend()

    plt.show()
    
class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.2):
        super().__init__()
        # Load a pre-trained model without the classifier
        self.base_model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=0)
        
        # Freeze all parameters of the pre-trained model
        for param in self.base_model.parameters():
            param.requires_grad = False

        # Get the number of features with the classifier removed
        features_num = self.base_model.num_features
        
        # Add custom layers
        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.batch_norm1 = nn.BatchNorm1d(features_num)
        # Add several fully connected layers
        self.fc1 = nn.Linear(features_num, features_num // 2)
        self.batch_norm2 = nn.BatchNorm1d(features_num // 2)
        self.fc2 = nn.Linear(features_num // 2, features_num // 4)
        self.batch_norm3 = nn.BatchNorm1d(features_num // 4)
        
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(features_num // 4, num_classes)

    def forward(self, x):
        x = self.base_model.forward_features(x) # features from the base model
        
        x = self.global_avg_pool(x).view(x.size(0), -1) # global average pooling
        
        x = self.batch_norm1(x) # batch normalization
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)

        x = self.classifier(x)
        return x
    
# Get parameters of the classifier only 
# since parameters of the pre-trained model are frozen
def get_all_layers(module):
    for layer in module.children():
        if list(layer.children()):  # fetch its children if it is a nested module
            yield from get_all_layers(layer)
        else:
            yield layer

In [ ]:
# Create an instance of the custom model
# model = CustomEfficientNet(num_classes=5, dropout_rate=0.2)

model = timm.create_model(MODEL_NAME, pretrained=True, num_classes=5)

# Get all layers as a list
# all_layers = list(get_all_layers(model.base_model))

# Compute the number of layers to unfreeze (e.g., the latter half of the model's layers)
# layers_to_unfreeze = len(all_layers) // 2

# Unfreeze the latter half of the layers
# for layer in all_layers[-layers_to_unfreeze:]:
#     for param in layer.parameters():
#         param.requires_grad = False

weights = torch.tensor(weights, dtype=torch.float32)
print(weights)

# Create a weighted CrossEntropyLoss
criterion = nn.CrossEntropyLoss(weight=weights.to(device))
# Define an optimizer
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

lr_scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=4, verbose=True)

device = torch.device("cuda:0") # device configuration

# Train the model
history = train_model(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, EPOCHS, device)

# Plot the metrics
plot_metrics(history)